In [4]:
from astropy.table import Table

import glob

import numpy as np

In [12]:
master_table = Table.read('../master_file_vflag_10.txt', format='ascii.ecsv')

['MaNGA_plate', 'MaNGA_fiberID', 'NSA_plate', 'NSA_fiberID', 'NSA_MJD', 'NSA_index', 'NSA_RA', 'NSA_DEC', 'NSA_ba', 'NSA_phi', 'NSA_redshift', 'NSA_Mstar', 'vflag', 'avg_v_max', 'avg_v_max_sigma', 'avg_alpha', 'avg_alpha_sigma', 'avg_chi_square_rot', 'pos_v_max', 'pos_v_max_sigma', 'pos_alpha', 'pos_alpha_sigma', 'pos_chi_square_rot', 'neg_v_max', 'neg_v_max_sigma', 'neg_alpha', 'neg_alpha_sigma', 'neg_chi_square_rot', 'center_flux', 'center_flux_error', 'Mtot', 'Mtot_error', 'Mdark', 'Mdark_error', 'Mstar', 'Mdark_Mstar_ratio', 'Mdark_Mstar_ratio_error', 'rabsmag', 'avg_r_turn', 'avg_r_turn_sigma', 'pos_r_turn', 'pos_r_turn_sigma', 'neg_r_turn', 'neg_r_turn_sigma', 'Z12logOH', 'logNO', 't3', 'BPT', 'curve_used', 'points_cut', 'avg_chi_square_ndf', 'pos_chi_square_ndf', 'neg_chi_square_ndf', 'index', 'Z12logOH_O3N2', 'Z12logOH_N2O2', 'Z12logOH_N2']


## Galaxies with rotation curves

In [12]:
boolean = np.logical_or(master_table['curve_used'] == 'non', master_table['curve_used'] == 'none')

rotCurve_galaxies = master_table[np.logical_not(boolean)]

print('Galaxies in MaNGA DR15:', len(master_table))
print('Galaxies with rotation curves:', len(rotCurve_galaxies))

Galaxies in MaNGA DR15: 4815
Galaxies with rotation curves: 3371


## Galaxies in each environment

In [13]:
vboolean = rotCurve_galaxies['vflag'] == 1
wboolean = rotCurve_galaxies['vflag'] == 0

void_table = rotCurve_galaxies[vboolean]
wall_table = rotCurve_galaxies[wboolean]
edge_table = rotCurve_galaxies[np.logical_not(np.logical_or(vboolean, wboolean))]

print('Void galaxies with rotation curves:', len(void_table))
print('Wall galaxies with rotation curves:', len(wall_table))
print('Edge galaxies with rotation curves:', len(edge_table))

Void galaxies with rotation curves: 930
Wall galaxies with rotation curves: 1255
Edge galaxies with rotation curves: 1186


## Galaxies by magnitude

In [15]:
print('Dwarf void galaxies:', sum(void_table['rabsmag'] >= -17))
print('Dwarf wall galaxies:', sum(wall_table['rabsmag'] >= -17))

print('-17:-18 void galaxies:', sum( np.logical_and( void_table['rabsmag'] < -17, void_table['rabsmag'] >= -18)))
print('-17:-18 wall galaxies:', sum( np.logical_and( wall_table['rabsmag'] < -17, wall_table['rabsmag'] >= -18)))

print('-18:-19 void galaxies:', sum( np.logical_and( void_table['rabsmag'] < -18, void_table['rabsmag'] >= -19)))
print('-18:-19 wall galaxies:', sum( np.logical_and( wall_table['rabsmag'] < -18, wall_table['rabsmag'] >= -19)))

print('-19:-20 void galaxies:', sum( np.logical_and( void_table['rabsmag'] < -19, void_table['rabsmag'] >= -20)))
print('-19:-20 wall galaxies:', sum( np.logical_and( wall_table['rabsmag'] < -19, wall_table['rabsmag'] >= -20)))

print('Void galaxies brighter than -20:', sum( void_table['rabsmag'] < -20))
print('Wall galaxies brighter than -20:', sum( wall_table['rabsmag'] < -20))

Dwarf void galaxies: 39
Dwarf wall galaxies: 27
-17:-18 void galaxies: 115
-17:-18 wall galaxies: 146
-18:-19 void galaxies: 256
-18:-19 wall galaxies: 312
-19:-20 void galaxies: 272
-19:-20 wall galaxies: 325
Void galaxies brighter than -20: 248
Wall galaxies brighter than -20: 445


## Fraction of masked spaxels

In [17]:
sum_frac_masked_spaxels = 0
N_gal = 0

for file in glob.iglob('../rot_curve_data_files/*_gal_stat_data.txt'):
    
    # Extract plate, fiberID
    file_name = file[24:]
    components = file_name.split('_')
    plate, fiberID = components[0].split('-')
    
    # Find galaxy in master_table
    plate_indices = np.argwhere(master_table['MaNGA_plate'] == int(plate))
    fiber_idx = np.where(master_table['MaNGA_fiberID'][plate_indices] == int(fiberID))
    gal_idx = plate_indices[fiber_idx]
    
    if master_table['curve_used'][gal_idx] != 'non' and master_table['curve_used'][gal_idx] != 'none':
        
        # Read in data
        gal_stats = Table.read(file, format='ascii.ecsv')

        # Update number of galaxies
        N_gal += 1

        # Update sum of the fraction of the number of masked spaxels
        sum_frac_masked_spaxels += gal_stats['frac_masked_spaxels'][0]
        
        
print('Average number of masked spaxels:', sum_frac_masked_spaxels/N_gal)

Average number of masked spaxels: 0.212001870176713
